In [37]:
import pandas as pd
import numpy as np
import re

In [38]:
#### import johnson et al. Nature 2023 data ####
jhonson_file = '/Users/smuellerdott/Documents/kinase_review/41586_2022_5575_MOESM5_ESM.csv'
jhonson_mapping = '/Users/smuellerdott/Documents/kinase_review/41586_2022_5575_MOESM3_ESM.csv'

In [39]:
# read jhhonson table
jhonson_df = pd.read_csv(jhonson_file, sep=';')
jhonson_df.columns = jhonson_df.columns.str.replace(r' +', '_', regex=True)
int_columns = ['Uniprot_Primary_Accession', 'Phosphosite', 'median_percentile', 'promiscuity_index']
percentile_columns = [i for i in jhonson_df.columns if i.endswith('percentile') and i != 'median_percentile']
int_columns = int_columns + percentile_columns
filt_jhonson_df = jhonson_df[int_columns].copy()

In [40]:
# read mapping table
jhonson_mapping_df = pd.read_csv(jhonson_mapping, sep=';', encoding='latin-1')
jhonson_mapping_df = jhonson_mapping_df.iloc[:, :2].drop_duplicates()
jhonson_mapping_df.columns = ['uniprot_id', 'matrix_name']

In [41]:
filt_jhonson_df

,Uniprot_Primary_Accession,Phosphosite,median_percentile,promiscuity_index,AAK1_percentile,ACVR2A_percentile,ACVR2B_percentile,AKT1_percentile,AKT2_percentile,AKT3_percentile,...,VRK1_percentile,VRK2_percentile,WNK1_percentile,WNK3_percentile,WNK4_percentile,YANK2_percentile,YANK3_percentile,YSK1_percentile,YSK4_percentile,ZAK_percentile
0,A0A075B6Q4,S24,"37,32",34,"0,47","74,44","83,75","40,8","33,48","33,87",...,"30,69","8,06","52,7","62,29","40,63","32,83","31,77","16,17","65,22","38,69"
1,A0A075B6Q4,S35,"21,65",10,"22,55","79,51","83,59","14,92","14,63","18,58",...,"13,28","8,57","15,44","26,16","11,66","54,85","40,65","2,34","14,99","0,78"
2,A0A075B6Q4,S57,"18,62",10,"4,22","90,58","93,27","12,36","8,18","6,02",...,"11,14","3,85","13,65","37,74","12,04","93,6","66,93","14,6","61,21","47,62"
3,A0A075B6Q4,S68,"65,75",42,"52,67","95,24","94,46","56,05","55,9","56,53",...,"71,79","40,39","51,63","85,37","78,68","91,31","73,52","76,66","90,57","82,24"
4,A0A075B6Q4,S71,"46,03",11,"25,22","60,68","69,21","56,89","29,8","38,44",...,"88,47","50,7","59,56","91,74","73,35","83,93","53,65","74,42","53,19","74,98"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89779,V9GYY5,S132,"27,76",40,"16,32","98,79","97,13","7,74","16,31","10,53",...,"14,29","5,8","13,14","16,25","22,77","91,98","62,89","30,3","58,9","73,59"
89780,V9GYY5,S133,"39,19",25,"9,79","90,07","88,92","39,19","46,19","51,58",...,"17,61","7,89","33,13","36,25","24,31","96,87","79,92","30,82","69,93","66,14"
89781,V9GYY5,T117,"30,68",3,"91,17","32,24","37,75","24,25","32,52","16,59",...,"75,89","91,58","25,13","12,59","32,54","1,84","1,28","45,06","19,33","13,55"
89782,V9GYY5,T134,"27,05",20,"68,48","91,77","93,71","17,85","9,86","13,19",...,"84,39","52,36","3,52","8,11","9,22","55,24","24,5","69,43","87,61","79,16"


In [42]:
jhonson_mapping_df

,uniprot_id,matrix_name
0,Q2M2I8,AAK1
1,Q04771,ALK2
2,P36896,ALK4
3,P27037,ACVR2A
4,Q13705,ACVR2B
...,...,...
298,Q99986,VRK1
299,Q86Y07,VRK2
300,Q9H4A3,WNK1
301,Q9BYP7,WNK3


In [43]:
# pivot longer all the percentile columns
long_jhonson_df = filt_jhonson_df.melt(id_vars=['Uniprot_Primary_Accession', 'Phosphosite', 'median_percentile', 'promiscuity_index'], value_vars=percentile_columns, var_name='kinase', value_name='percentile')
long_jhonson_df['percentile'] = pd.to_numeric(long_jhonson_df['percentile'], errors='coerce')

In [44]:
# filter percentile values to be above 90
filt_long_jhonson_df = long_jhonson_df[long_jhonson_df['percentile'] >= 90].copy()
filt_long_jhonson_df['kinase'] = filt_long_jhonson_df['kinase'].str.replace('_percentile', '', regex=True)

In [45]:
filt_long_jhonson_df

,Uniprot_Primary_Accession,Phosphosite,median_percentile,promiscuity_index,kinase,percentile
1852,K7ELQ4,T53,"37,29",21,AAK1,92.0
3428,O14874,T32,"67,32",51,AAK1,93.0
4042,O15066,T732,"31,63",6,AAK1,94.0
5749,O43598,S123,"78,69",77,AAK1,91.0
9245,O76080,T8,"31,98",13,AAK1,93.0
...,...,...,...,...,...,...
27200284,Q9UPW8,S443,"65,78",45,ZAK,94.0
27201543,Q9Y2J2,S1068,"57,53",20,ZAK,97.0
27201736,Q9Y2L6,T529,"64,61",94,ZAK,100.0
27203168,Q9Y4L1,S461,"62,27",57,ZAK,97.0


In [46]:
# left join with the mapping table using kinase = matrix_name
merged_jhonson_df = filt_long_jhonson_df.merge(jhonson_mapping_df, left_on='kinase', right_on='matrix_name', how='left')

In [47]:
merged_jhonson_df

,Uniprot_Primary_Accession,Phosphosite,median_percentile,promiscuity_index,kinase,percentile,uniprot_id,matrix_name
0,K7ELQ4,T53,"37,29",21,AAK1,92.0,Q2M2I8,AAK1
1,O14874,T32,"67,32",51,AAK1,93.0,Q2M2I8,AAK1
2,O15066,T732,"31,63",6,AAK1,94.0,Q2M2I8,AAK1
3,O43598,S123,"78,69",77,AAK1,91.0,Q2M2I8,AAK1
4,O76080,T8,"31,98",13,AAK1,93.0,Q2M2I8,AAK1
...,...,...,...,...,...,...,...,...
29771,Q9UPW8,S443,"65,78",45,ZAK,94.0,Q9NYL2,ZAK
29772,Q9Y2J2,S1068,"57,53",20,ZAK,97.0,Q9NYL2,ZAK
29773,Q9Y2L6,T529,"64,61",94,ZAK,100.0,Q9NYL2,ZAK
29774,Q9Y4L1,S461,"62,27",57,ZAK,97.0,Q9NYL2,ZAK


In [48]:
# simplify the table, by keeping the 'Uniprot_Primary_Accession', 'Phosphosite' columns, and the 'uniprot_id' column
simplified_jhonson_df = merged_jhonson_df[['Uniprot_Primary_Accession', 'Phosphosite', 'matrix_name', 'percentile']].copy()
simplified_jhonson_df['source'] = simplified_jhonson_df['matrix_name']
simplified_jhonson_df['target'] = simplified_jhonson_df['Uniprot_Primary_Accession'] + '_' + simplified_jhonson_df['Phosphosite']
simplified_jhonson_df = simplified_jhonson_df[['source', 'target', 'percentile']].copy().rename(columns={'percentile': 'weight'})

In [49]:
simplified_jhonson_df

,source,target,weight
0,AAK1,K7ELQ4_T53,92.0
1,AAK1,O14874_T32,93.0
2,AAK1,O15066_T732,94.0
3,AAK1,O43598_S123,91.0
4,AAK1,O76080_T8,93.0
...,...,...,...
29771,ZAK,Q9UPW8_S443,94.0
29772,ZAK,Q9Y2J2_S1068,97.0
29773,ZAK,Q9Y2L6_T529,100.0
29774,ZAK,Q9Y4L1_S461,97.0


In [50]:
unique_sources = len(simplified_jhonson_df['source'].unique())
print(unique_sources)

303
